In [ ]:
pip install datasets

In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("ninadn/indian-legal")
print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['Text', 'Summary'],
        num_rows: 7030
    })
    test: Dataset({
        features: ['Text', 'Summary'],
        num_rows: 100
    })
})


In [ ]:
from transformers import AutoTokenizer

# Load a pre-trained tokenizer (e.g., BART)
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the dataset
def preprocess_function(examples):
    # Ensure inputs are strings
    inputs = [str(doc) for doc in examples["Text"]]  # Convert to string if necessary
    targets = [str(summary) for summary in examples["Summary"]]  # Convert to string if necessary

    # Tokenize inputs (text) and targets (summary)
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    # Add labels to the model inputs
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing to the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
# Rename the "Test" split to "Validation" for clarity
tokenized_dataset["validation"] = tokenized_dataset.pop("test")

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Load the pre-trained model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./legal-summarization-model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,  # Adjust based on your GPU memory
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    fp16=True,
    report_to="none" # Enable mixed precision for faster training on your GPU
)

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-5-3a87bee13a40>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,1.575000,1.762289
2,1.310900,1.710994


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


In [ ]:
import shutil
from google.colab import files

# Save the model and tokenizer
model.save_pretrained("./legal-summarization-model")
tokenizer.save_pretrained("./legal-summarization-model")

# Compress the model directory
shutil.make_archive("legal-summarization-model", 'zip', "./legal-summarization-model")

# Download the zip file to your local machine
from google.colab import files
files.download("legal-summarization-model.zip")